In [8]:

#importing libraries for model evaluation and algorithms
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import collections
import nltk
from sklearn import preprocessing
from nltk.tokenize import sent_tokenize, word_tokenize
# Packages for data preparation
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords
from keras.preprocessing.text import Tokenizer
from keras.utils.np_utils import to_categorical
from sklearn.preprocessing import LabelEncoder
#dl libraraies
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D , MaxPool2D , Flatten , Dropout , BatchNormalization,Reshape,Dot,Concatenate,Add,Lambda
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ReduceLROnPlateau
import cv2
from tensorflow.keras.optimizers import Adam,SGD,Adagrad,Adadelta,RMSprop
import os
from tensorflow.keras.models import Model
from tensorflow.keras.regularizers import l2
from keras.layers.recurrent import LSTM
from keras.preprocessing.sequence import pad_sequences
# specifically for deeplearning.
from tensorflow.keras.layers import Dropout, Flatten,Activation,Input,Embedding
from tensorflow.keras.layers import Conv2D, MaxPooling2D, BatchNormalization
# Packages for modeling
from keras import models
from keras import layers
from keras import regularizers

In [9]:
#reading the dataframe
df=pd.read_csv('Tweets.csv')
df.head()

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,570306133677760513,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada)
1,570301130888122368,positive,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada)
2,570301083672813571,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I n...,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada)
3,570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica it's really aggressive to blast...,NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada)
4,570300817074462722,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing...,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada)


In [10]:
#removing the neutral sentiments considering that positive and negative sentiments matter more 
df=df[df['airline_sentiment']!='neutral']
#now since neutral elements are deleted so I need to reset the indices
df.reset_index(inplace=True,drop=True)
#positive sentiments to 1 and negative to 0 
def partition(x):
    if x =='positive':
        return 1
    return 0
actualSentiment = df['airline_sentiment']
positiveNegative = actualSentiment.map(partition) 
df['Sentiment'] = positiveNegative
df['Sentiment'].value_counts() 

0    9178
1    2363
Name: Sentiment, dtype: int64

In [12]:
#Setting parameters which will be used throughout
num_words = 15000  # Parameter indicating the number of words we'll put in the dictionary
val_size = 1000  # Size of the validation set
epochs = 20  # Number of epochs we usually start to train with
batch_size = 512  # Size of the batches used in the mini-batch gradient descent
#Taking only two columns since it's a sentiment analysis

In [14]:
#tweets conssits of every document as an array of tokenized words which are later appended to docs 
tweets=[word_tokenize(tweet) for tweet in df['text']]
docs=[]
for j in range(0,len(tweets)):
    docs.append(tweets[j])

In [15]:
#stops included both the stopwords and punctuations
from nltk.corpus import stopwords
import string
stops = set(stopwords.words('english'))
punctuations = list(string.punctuation)
not_list = ["n't", "not", "no"]
stops.update(punctuations)
stops.update(not_list)

In [16]:
#to get the simple pos(part of speech) tag
from nltk.corpus import wordnet
def get_simple_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

In [17]:
#to get the pos tag for a word
from nltk import pos_tag
# now we are going to clean our data 
# we will remove stopwords and punctuations and lemmatize each document
from nltk.stem import WordNetLemmatizer
lemmatizer=WordNetLemmatizer()
def clean(words):
    output=[]
    for word in words:
        if word.lower() not in stops or word.lower() in not_list:
            pos=pos_tag(word)
            clean_word=lemmatizer.lemmatize(word,pos=get_simple_pos(pos[0][1]))
            output.append(clean_word.lower())
    str1=" ".join(output).encode('utf-8')        
    return str1
docs=[ clean(doc) for doc in docs]      

In [ ]:
#docs was the cleaned lemmatized text which has been appended into the dataframe for further use
df['CleanedText']=docs
df['CleanedText']=df['CleanedText'].str.decode("utf-8")
def remove_mentions(input_text):
        return re.sub(r'@\w+', '', input_text)
df.text = df.CleanedText.apply(remove_mentions)
df.head()

In [19]:
#taking only two columns in the dataframe
df=df[['CleanedText','Sentiment']]
df.head()

,CleanedText,Sentiment
0,virginamerica plus 've added commercial experi...,1
1,virginamerica 's really aggressive blast obnox...,0
2,virginamerica 's really big bad thing,0
3,virginamerica seriously would pay 30 flight se...,0
4,virginamerica yes nearly every time fly vx “ e...,1


In [20]:
#taking variables to be used for train test split as X,y
X,Y=df['CleanedText'].values,pd.get_dummies(df.Sentiment.values)

In [21]:
#using tokenizers to create the tokens having no of words=15000(num_words)
tk = Tokenizer(num_words=num_words,
               filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',
               lower=True,
               split=" ")
#Complete data is tokenized to vectors and padding is done using zeros to match its length to the largest text in the dataset.
tk.fit_on_texts(X)
X = tk.texts_to_sequences(X)
X = pad_sequences(X)
#print(X[:2])
print('Fitted tokenizer on {} documents'.format(tk.document_count))
print('{} words in dictionary'.format(tk.num_words))
print('Top 5 most common words are:', collections.Counter(tk.word_counts).most_common(5))

Fitted tokenizer on 11541 documents
15000 words in dictionary
Top 5 most common words are: [('united', 3426), ('flight', 3321), ('usairways', 2651), ('americanair', 2465), ("n't", 1878)]


In [23]:
#train test split
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size=0.2, random_state=42)
print('# Train data samples:', X_train.shape)
print('# Test data samples:', X_test.shape)

# Train data samples: (9232, 27)
# Test data samples: (2309, 27)


In [24]:
#getting validation data as a part of training data
X_train_rest, X_valid, Y_train_rest, Y_valid = train_test_split(X_train,Y_train, test_size=0.1, random_state=37)
print('Shape of validation set:',X_valid.shape)

Shape of validation set: (924, 27)


In [25]:
#Function defined to test the models in the test set
def test_model(model, epoch_stop):
    model.fit(X_test
              , Y_test
              , epochs=epoch_stop
              , batch_size=batch_size
              , verbose=0)
    results = model.evaluate(X_test, Y_test)
    
    return results

In [26]:
embed_dim = 128 #dimension of the word embedding vector for each word in a sequence 
lstm_out = 196  #no of lstm layers
lstm_model = Sequential()
lstm_model.add(Embedding(num_words, embed_dim,input_length = X_train.shape[1]))
#Adding dropout
lstm_model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
#Adding a regularized dense layer
lstm_model.add(layers.Dense(32,kernel_regularizer=regularizers.l2(0.001),activation='relu'))
lstm_model.add(layers.Dropout(0.5))
lstm_model.add(Dense(2,activation='softmax'))
lstm_model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(lstm_model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 27, 128)           1920000   
_________________________________________________________________
lstm (LSTM)                  (None, 196)               254800    
_________________________________________________________________
dense (Dense)                (None, 32)                6304      
_________________________________________________________________
dropout (Dropout)            (None, 32)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 66        
Total params: 2,181,170
Trainable params: 2,181,170
Non-trainable params: 0
_________________________________________________________________
None


In [27]:
#model trained on the training data and taking validation data into account to avoid overfitting for 4 epochs 
history_LSTM=lstm_model.fit(X_train_rest, Y_train_rest, epochs = 4, batch_size=batch_size,validation_data=(X_valid, Y_valid),verbose = 1)

Epoch 1/4
17/17 [==============================] - 15s 729ms/step - loss: 0.6817 - accuracy: 0.7165 - val_loss: 0.5371 - val_accuracy: 0.7955
Epoch 2/4
17/17 [==============================] - 12s 689ms/step - loss: 0.5103 - accuracy: 0.7950 - val_loss: 0.3823 - val_accuracy: 0.8517
Epoch 3/4
17/17 [==============================] - 12s 692ms/step - loss: 0.3421 - accuracy: 0.8832 - val_loss: 0.2673 - val_accuracy: 0.9037
Epoch 4/4
17/17 [==============================] - 12s 690ms/step - loss: 0.2012 - accuracy: 0.9451 - val_loss: 0.2197 - val_accuracy: 0.9307


In [28]:
#prediction by our lstm model on the test dataset
lstm_results = test_model(lstm_model, 3)
print('/n')
print('Test accuracy of lstm model: {0:.2f}%'.format(lstm_results[1]*100))

73/73 [==============================] - 1s 16ms/step - loss: 0.1186 - accuracy: 0.9701
/n
Test accuracy of lstm model: 97.01%


##Now we will see the predictions by our model on sample movie reviews

***Class 1 is positive class and class 0 is negative class***

In [31]:
#First the sample text is a positive review, so lets see its prediction by our model
sample_text = ('The movie was good. Everything was so awesome. Highly recommended.')
#converting into sequence because my model is trained on such a sequence
sample = tk.texts_to_sequences([sample_text])
sample = pad_sequences(sample)
sample.shape

(1, 10)

In [32]:
#In training 27 was the no of tokens for each tweet so padding by zeros so that model can accept it
k=np.zeros((1,27))
k[0,-10:]=sample
k

array([[   0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
           0.,    0.,    0.,    0.,    0.,    0.,    0.,    0., 2423.,
        1348., 7534.,   89.,  456., 7534., 2175.,  190., 2243., 3829.]])

In [33]:
#Prediction by model and as expected we get the class as 1 by almost 93%
predictions = lstm_model.predict_classes(np.array(k))
prediction_probability=lstm_model.predict(np.array(k))
print(predictions)
print(prediction_probability[0])

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


[1]
[0.07073332 0.92926663]


In [34]:
#Secondly the sample text is a negative review, so lets see its prediction by our model
sample_text = ('The movie was pathetic and terrible.Nothing was making any sense. Highly unacceptable.')
#converting into sequence because my model is trained on such a sequence
sample = tk.texts_to_sequences([sample_text])
sample = pad_sequences(sample)
sample.shape

(1, 13)

In [35]:
#In training 27 was the no of tokens for each tweet so padding by zeros so that model can accept it
k=np.zeros((1,27))
k[0,-13:]=sample
k

array([[    0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
            0.,     0.,     0.,     0.,     0.,     0.,  2423.,  1348.,
         7534.,   845.,  1778.,   223.,   153.,  7534.,   228., 12070.,
          656.,  2243.,   257.]])

In [36]:
#Prediction by model and as expected we get the class as 0 by 99.99% probability. 
predictions = lstm_model.predict_classes(np.array(k))
prediction_probability=lstm_model.predict(np.array(k))
print(predictions)
print(prediction_probability[0])

[0]
[9.999926e-01 7.404823e-06]


/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '
